# Textured Cube Example

This notebook demonstrates applying a texture to all faces of a rotating cube using Matplotlib's 3D capabilities. Provide your own image file named `texture.jpg` in the repository root to customise the texture; a random one is generated if none is found.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

if os.path.exists("texture.jpg"):
    texture = plt.imread("texture.jpg")
    if texture.dtype == np.uint8:
        texture = texture.astype(np.float32) / 255.0
else:
    texture = np.random.rand(256, 256, 3)

size = min(texture.shape[0], texture.shape[1])
texture = texture[:size, :size]
max_size = 256
step = max(1, size // max_size)
texture = texture[::step, ::step]
M, N, _ = texture.shape

u = np.linspace(-0.5, 0.5, N)
v = np.linspace(-0.5, 0.5, M)
U, V = np.meshgrid(u, v)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.axis('off')
ax.set_box_aspect([1, 1, 1])

faces = []
faces.append(ax.plot_surface(U, 0.5*np.ones_like(U), V, rstride=1, cstride=1, facecolors=texture))
faces.append(ax.plot_surface(U, -0.5*np.ones_like(U), V, rstride=1, cstride=1, facecolors=texture))
faces.append(ax.plot_surface(U, V, 0.5*np.ones_like(U), rstride=1, cstride=1, facecolors=texture))
faces.append(ax.plot_surface(U, V, -0.5*np.ones_like(U), rstride=1, cstride=1, facecolors=texture))
faces.append(ax.plot_surface(0.5*np.ones_like(U), U, V, rstride=1, cstride=1, facecolors=texture))
faces.append(ax.plot_surface(-0.5*np.ones_like(U), U, V, rstride=1, cstride=1, facecolors=texture))

def update(angle):
    ax.view_init(30, angle)
    return faces

ani = FuncAnimation(fig, update, frames=np.linspace(0, 360, 120), interval=1000/60, blit=True)
HTML(ani.to_jshtml())
